In [29]:
#1 Object: 개체별 데이터셋 생성 및 분리.
#1 Detail: 한우와 홀스타인 데이터를 train/val로 분리하여 저장.
'''
hanwoo/
│  ├── annotations/          # XML 어노테이션 파일들
│  │   └── .xml
│  └── muzzle/
│      └── unit/
│          └── .jpg         # 이미지 파일들

holstein/
│  ├── annotations/          # XML 어노테이션 파일들
│  │   └── .xml
│  └── muzzle/
│      └── unit/
│          └── .jpg         # 이미지 파일들

datasets/
├── hanwoo/
│   ├── train/
│   │   ├── images/         # 학습용 이미지 파일들
│   │   └── labels/         # 학습용 라벨 파일들
│   └── val/
│       ├── images/         # 검증용 이미지 파일들
│       └── labels/         # 검증용 라벨 파일들
├── holstein/
    ├── train/
    │   ├── images/         # 학습용 이미지 파일들
    │   └── labels/         # 학습용 라벨 파일들
    └── val/
        ├── images/         # 검증용 이미지 파일들
        └── labels/         # 검증용 라벨 파일들
'''
import os
import random
import shutil
import xml.etree.ElementTree as ET
import re
from tqdm import tqdm  # 진행 상황 표시를 위한 라이브러리

#1-1 Object: YOLO 데이터셋 디렉토리 생성
#1-1 Detail: 한우와 홀스타인 데이터를 저장할 train/val 디렉토리 생성
BASE_DIR = r"F:\imageDB\cattleDB"  # 실제 데이터 경로로 변경
HANWOO_IMAGE_DIR = os.path.join(BASE_DIR, "hanwoo/muzzle")
HANWOO_ANNOTATION_DIR = os.path.join(BASE_DIR, "hanwoo/annotations")
HOLSTEIN_IMAGE_DIR = os.path.join(BASE_DIR, "holstein/muzzle")
HOLSTEIN_ANNOTATION_DIR = os.path.join(BASE_DIR, "holstein/annotations")
OUTPUT_DIR = os.path.join(BASE_DIR, "datasets")
CLASSES = ['muzzle']  # 클래스 목록

#1-1-1 YOLO 데이터셋 디렉토리 생성 함수
def create_dirs(base_output_dir, categories):
    for category in categories:
        os.makedirs(os.path.join(base_output_dir, category, "train/images"), exist_ok=True)
        os.makedirs(os.path.join(base_output_dir, category, "train/labels"), exist_ok=True)
        os.makedirs(os.path.join(base_output_dir, category, "val/images"), exist_ok=True)
        os.makedirs(os.path.join(base_output_dir, category, "val/labels"), exist_ok=True)

#1-1-2 디렉토리 생성
create_dirs(OUTPUT_DIR, ["hanwoo", "holstein"])  # 한우와 홀스타인 카테고리별로 디렉토리 생성
print("YOLO 데이터셋 디렉토리 생성 완료!")

#1-2 Object: XML -> YOLO 포맷 변환
#1-2 Detail: XML 어노테이션 파일을 YOLO 포맷으로 변환하여 .txt 라벨 파일을 생성
def convert_xml_to_yolo(xml_path, output_path, classes):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)

        yolo_labels = []
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in classes:
                continue
            class_id = classes.index(class_name)
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # YOLO 포맷으로 변환 (정규화)
            x_center = (xmin + xmax) / 2.0 / width
            y_center = (ymin + ymax) / 2.0 / height
            w = (xmax - xmin) / float(width)
            h = (ymax - ymin) / float(height)
            yolo_labels.append(f"{class_id} {x_center} {y_center} {w} {h}")

        if yolo_labels:
            with open(output_path, 'w') as f:
                f.write("\n".join(yolo_labels))
        else:
            print(f"Warning: {xml_path}에는 라벨이 없습니다.")
    except FileNotFoundError:
        print(f"Error: XML 파일 {xml_path}을 찾을 수 없습니다.")
    except ET.ParseError:
        print(f"Error: {xml_path} 파일 파싱 중 오류 발생.")
    except Exception as e:
        print(f"Error converting {xml_path} to YOLO format: {e}")

#1-3 Object: 이미지 파일 목록 생성 및 XML 파일 매칭
#1-3 Detail: 이미지 파일을 찾고, 각 이미지에 매칭되는 XML 파일을 확인하여 처리
def convert_xml_to_yolo(xml_path, output_path, classes):
    """
    XML 파일을 YOLO 포맷으로 변환하여 .txt 라벨 파일을 생성
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # 이미지 크기 가져오기
        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)

        yolo_labels = []

        # 'muzzle' 클래스 처리
        for obj in root.findall('object'):
            class_name = obj.find('name').text.strip()  # 객체 이름을 추출

            # 원래 클래스 이름을 'muzzle'로 바꿈
            if class_name != 'muzzle':
                class_name = 'muzzle'  # 여기서 'muzzle'로 바꿔줌
            
            class_id = classes.index(class_name)  # 'muzzle'은 클래스 0
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # YOLO 포맷으로 변환 (정규화)
            x_center = (xmin + xmax) / 2.0 / width
            y_center = (ymin + ymax) / 2.0 / height
            w = (xmax - xmin) / float(width)
            h = (ymax - ymin) / float(height)
            yolo_labels.append(f"{class_id} {x_center} {y_center} {w} {h}")

        if yolo_labels:
            with open(output_path, 'w') as f:
                f.write("\n".join(yolo_labels))
        else:
            print(f"Warning: {xml_path}에는 'muzzle' 클래스가 없거나 라벨이 없습니다.")
    except FileNotFoundError:
        print(f"Error: XML 파일 {xml_path}을 찾을 수 없습니다.")
    except ET.ParseError:
        print(f"Error: {xml_path} 파일 파싱 중 오류 발생.")
    except Exception as e:
        print(f"Error converting {xml_path} to YOLO format: {e}")

 #1-3-1 이미지 파일 목록 생성 함수
def get_image_files(image_dir):
    image_files = []
    for folder in os.listdir(image_dir):
        folder_path = os.path.join(image_dir, folder)
        if os.path.isdir(folder_path):
            for img in os.listdir(folder_path):
                if img.endswith('.jpg'):
                    image_files.append(os.path.join(folder_path, img))
    return image_files

def split_and_convert_data(image_dir, annotation_dir, output_dir, category, classes, train_ratio=0.8):
    train_image_dir = os.path.join(output_dir, category, "train/images")
    train_label_dir = os.path.join(output_dir, category, "train/labels")
    val_image_dir = os.path.join(output_dir, category, "val/images")
    val_label_dir = os.path.join(output_dir, category, "val/labels")

    image_files = get_image_files(image_dir)
    random.shuffle(image_files)
    split_idx = int(len(image_files) * train_ratio)
    train_images = image_files[:split_idx]
    val_images = image_files[split_idx:]

    # train 데이터 처리
    for img_path in tqdm(train_images, desc=f"Processing {category} train data"):
        img_name = os.path.basename(img_path)
        xml_file = find_matching_xml(img_name, annotation_dir)

        if xml_file:
            xml_path = os.path.join(annotation_dir, xml_file)
            output_label_path = os.path.join(train_label_dir, os.path.splitext(img_name)[0] + '.txt')
            if os.path.exists(xml_path) and xml_path.endswith('.xml'):
                try:
                    shutil.copy(img_path, train_image_dir)  # 이미지 복사
                    print(f"Copying {img_path} to {train_image_dir}")  # 디버깅 출력
                    convert_xml_to_yolo(xml_path, output_label_path, classes)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
            else:
                print(f"Warning: {xml_path}가 없거나 형식이 올바르지 않습니다.")
        else:
            print(f"Warning: {img_path}에 해당하는 XML 파일을 찾을 수 없습니다.")
            
#1-4 Object: 이미지 파일과 XML 파일 매칭 (더욱 robust하게)
#1-4 Detail: 이미지 이름을 기반으로 가장 적합한 XML 파일을 찾음
def find_matching_xml(image_name, annotation_dir):
    image_base_name = os.path.splitext(image_name)[0]
    best_match = None
    max_similarity = 0

    for xml_file in os.listdir(annotation_dir):
        if not xml_file.endswith('.xml'):
            continue
        xml_base_name = os.path.splitext(xml_file)[0]
        similarity = len(re.findall(r'\d+', image_base_name))  # 숫자 부분만 비교
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = xml_file
    return best_match

#1-5 Object: 데이터 분리 및 변환
#1-5 Detail: 이미지를 train/val로 분리하고, YOLO 포맷으로 변환하여 저장
def split_and_convert_data(image_dir, annotation_dir, output_dir, category, classes, train_ratio=0.8):
    train_image_dir = os.path.join(output_dir, category, "train/images")
    train_label_dir = os.path.join(output_dir, category, "train/labels")
    val_image_dir = os.path.join(output_dir, category, "val/images")
    val_label_dir = os.path.join(output_dir, category, "val/labels")

    image_files = get_image_files(image_dir)
    random.shuffle(image_files)
    split_idx = int(len(image_files) * train_ratio)
    train_images = image_files[:split_idx]
    val_images = image_files[split_idx:]

    # train 데이터 처리
    for img_path in tqdm(train_images, desc=f"Processing {category} train data"):  # 진행 상태 표시
        img_name = os.path.basename(img_path)
        xml_file = find_matching_xml(img_name, annotation_dir)

        if xml_file:
            xml_path = os.path.join(annotation_dir, xml_file)
            output_label_path = os.path.join(train_label_dir, os.path.splitext(img_name)[0] + '.txt')
            if os.path.exists(xml_path) and xml_path.endswith('.xml'):
                try:
                    shutil.copy(img_path, train_image_dir)
                    convert_xml_to_yolo(xml_path, output_label_path, classes)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
            else:
                print(f"Warning: {xml_path}가 없거나 형식이 올바르지 않습니다. {img_path}에 매칭되는 xml파일을 확인해주세요.")
        else:
            print(f"Warning: {img_path}에 해당하는 XML 파일을 찾을 수 없습니다.")

    # val 데이터 처리
    for img_path in tqdm(val_images, desc=f"Processing {category} val data"):  # 진행 상태 표시
        img_name = os.path.basename(img_path)
        xml_file = find_matching_xml(img_name, annotation_dir)

        if xml_file:
            xml_path = os.path.join(annotation_dir, xml_file)
            output_label_path = os.path.join(val_label_dir, os.path.splitext(img_name)[0] + '.txt')
            if os.path.exists(xml_path) and xml_path.endswith('.xml'):
                try:
                    shutil.copy(img_path, val_image_dir)
                    convert_xml_to_yolo(xml_path, output_label_path, classes)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
            else:
                print(f"Warning: {xml_path}가 없거나 형식이 올바르지 않습니다. {img_path}에 매칭되는 xml파일을 확인해주세요.")
        else:
            print(f"Warning: {img_path}에 해당하는 XML 파일을 찾을 수 없습니다.")

#1-6 실행
create_dirs(OUTPUT_DIR, ["hanwoo", "holstein"])

split_and_convert_data(HANWOO_IMAGE_DIR, HANWOO_ANNOTATION_DIR, OUTPUT_DIR, "hanwoo", CLASSES)
split_and_convert_data(HOLSTEIN_IMAGE_DIR, HOLSTEIN_ANNOTATION_DIR, OUTPUT_DIR, "holstein", CLASSES)

#1-7 완료 메시지 출력
print("한우 및 홀스타인 데이터 train/val 분리 완료!")

YOLO 데이터셋 디렉토리 생성 완료!


Processing hanwoo train data:  27%|██▋       | 2010/7462 [02:49<05:15, 17.26it/s]  

In [23]:
#2 Object: YOLO 데이터 구성 파일 생성.
#2 Detail: YOLOv8 학습에 필요한 data.yaml 파일 생성.

import os

#2-1 data.yaml 파일 내용 정의
data_yaml_content = f"""
train: F:/imageDB/cattleDB/datasets/hanwoo/train/images  # 학습 이미지 경로
val: F:/imageDB/cattleDB/datasets/hanwoo/val/images      # 검증 이미지 경로

nc: 1  # 클래스 수 (비문)
names: ['muzzle']  # 클래스 이름
"""

#2-2 data.yaml 파일 저장 경로 설정
data_yaml_path = r"F:/imageDB/cattleDB/datasets/data.yaml"

#2-3 data.yaml 파일 생성
with open(data_yaml_path, 'w') as yaml_file:
    yaml_file.write(data_yaml_content)

#2-4 확인 메시지 출력
print(f"data.yaml 파일 생성 완료! 경로: {data_yaml_path}")


data.yaml 파일 생성 완료! 경로: F:/imageDB/cattleDB/datasets/data.yaml


In [25]:
#3 Object: YOLOv8 모델 학습
#3 Detail: 준비된 데이터로 YOLOv8 모델 학습 수행

#3-1 데이터셋 구성 파일 경로 설정
# data.yaml 파일의 경로를 설정합니다.
data_yaml_path = r"F:/imageDB/cattleDB/datasets/data.yaml"  # data.yaml 파일 경로

#3-2 YOLOv8 모델 초기화
# YOLOv8 모델을 Nano 버전으로 초기화합니다. (경량 모델)
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # YOLOv8 Nano 버전 (경량 모델)

#3-3 YOLOv8 모델 학습 수행
# 학습을 위해 YOLOv8 모델을 학습시킵니다. data.yaml 경로와 학습 파라미터를 설정합니다.
model.train(
    data=data_yaml_path,  # data.yaml 파일 경로
    epochs=20,             # 학습 에포크 수
    batch=16,              # 배치 크기
    imgsz=640,             # 입력 이미지 크기
    save=True,             # 모델 학습 결과 저장
    project="runs/train",  # 학습 결과 저장 위치
    name="muzzle_detector"  # 프로젝트 이름
)

#3-4 학습 완료 메시지 출력
print("YOLOv8 모델 학습 완료!")


New https://pypi.org/project/ultralytics/8.3.55 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.155  Python-3.11.9 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=F:/imageDB/cattleDB/datasets/data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=muzzle_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False

YOLOv8 모델 학습 완료!


In [24]:
#검정

import pandas as pd

# CSV 파일 경로 설정
csv_file_path = r"C:\Users\tj\Desktop\runs\train\muzzle_detector3\results.csv"

# CSV 파일 읽기
results_df = pd.read_csv(csv_file_path)

# 열 이름에서 공백 제거
results_df.columns = results_df.columns.str.strip()

# 마지막 epoch에서의 주요 성능 지표 출력
last_epoch = results_df.iloc[-1]

# 수정된 열 이름을 사용하여 성능 지표 출력
print(f"Final Epoch: {last_epoch['epoch']}")
print(f"Final Precision: {last_epoch['metrics/precision(B)']}")
print(f"Final Recall: {last_epoch['metrics/recall(B)']}")
print(f"Final mAP@0.5: {last_epoch['metrics/mAP50(B)']}")
print(f"Final mAP@0.5:0.95: {last_epoch['metrics/mAP50-95(B)']}")


Final Epoch: 19.0
Final Precision: 0.2899
Final Recall: 0.98285
Final mAP@0.5: 0.29503
Final mAP@0.5:0.95: 0.29503


Ultralytics YOLOv8.0.155  Python-3.11.9 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
Model summary (fused): 168 layers, 3005843 parameters, 12675 gradients, 8.1 GFLOPs

Dataset 'coco.yaml' images not found , missing path 'C:\Users\tj\Desktop\vsc\ubuntu\1workspace\win_server\datasets\coco\val2017.txt'
100%|██████████| 169M/169M [00:15<00:00, 11.2MB/s] 
Unzipping C:\Users\tj\Desktop\vsc\ubuntu\1workspace\win_server\datasets\coco2017labels-segments.zip to C:\Users\tj\Desktop\vsc\ubuntu\1workspace\win_server\datasets...: 100%|██████████| 122232/122232 [01:23<00:00, 1460.37file/s]
 Download failure, retrying 1/3 http://images.cocodataset.org/zips/train2017.zip...
 Download failure, retrying 2/3 http://images.cocodataset.org/zips/train2017.zip...
 Download failure, retrying 3/3 http://images.cocodataset.org/zips/train2017.zip...


ConnectionError:   Download failure for http://images.cocodataset.org/zips/train2017.zip. Retry limit reached.

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (615219566.py, line 1)